In [1]:
from datasets import load_dataset # Import dataset import function for hugging face
dataset = load_dataset("surrey-nlp/PLOD-CW") # import the coursework dataset from

In [8]:
# Dataset splits into 3 distinct parts:
# train, validation and test

# To access each call dataset[set_name]
# For instance dataset["train"]

# Each set has 3 features:
# tokens, pos_tags, ner_tags

# To access each features, call dataset[set_name][feature_name]
# for instance dataset["train"]["tokens"]

# print(dataset)
# print(dataset["train"]["tokens"][0])

In [10]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics

In [14]:
train_tokens = dataset["train"]["tokens"]
train_pos_tags = dataset["train"]["pos_tags"]
train_ner_tags = dataset["train"]["ner_tags"]

validation_tokens = dataset["validation"]["tokens"]
validation_pos_tags = dataset["validation"]["pos_tags"]
validation_ner_tags = dataset["validation"]["ner_tags"]

test_tokens = dataset["test"]["tokens"]
test_pos_tags = dataset["test"]["pos_tags"]
test_ner_tags = dataset["test"]["ner_tags"]

class DataItem:
    def __init__(self, tokens, pos, ner):
        self.tokens:list[str] = tokens
        self.pos:list[str] = pos
        self.ner:list[str] = ner

    def get_as_tuple(self) -> tuple:
        return (self.tokens, self.pos, self.ner)
    
train_data:list[DataItem] = []
for idx in range(len(train_tokens)):
    train_data.append(DataItem(train_tokens[idx], train_pos_tags[idx], train_ner_tags[idx]))

validation_data:list[DataItem] = []
for idx in range(len(validation_tokens)):
    train_data.append(DataItem(validation_tokens[idx], validation_pos_tags[idx], validation_ner_tags[idx]))

test_data:list[DataItem] = []
for idx in range(len(test_tokens)):
    train_data.append(DataItem(test_tokens[idx], test_pos_tags[idx], test_ner_tags[idx]))



In [22]:
def create_data_sents(data:list[DataItem]) -> list[tuple]:
    sent_list:list[tuple] = []
    for data_obj in data:
        data_tuple = data_obj.get_as_tuple()
        tokens = data_obj.tokens
        pos = data_obj.pos
        ner = data_obj.ner
        
        for data_idx in range(len(data_tuple)-1):
            sent_list.append((tokens[data_idx], pos[data_idx], ner[data_idx]))
    return sent_list

train_sents:list[tuple] = create_data_sents(train_data)
validation_sents:list[tuple] = create_data_sents(validation_data)
test_sents:list[tuple] = create_data_sents(test_data)